# Dynamic Binary Instrumentation

Dynamic binary instrumentation (DBI) allows to inspect and modify program behavior at runtime. In this chapter, we will be using Intel's Pin framework to instrument programs at runtime.

**Prerequisites**

* You should be familiar with the chapter on ["Mutation-based Fuzzing"](MutationFuzzer.ipynb)

## Synopsis
TBD

In [ ]:
import fuzzingbook_utils
from Fuzzer import ProgramRunner
from MutationFuzzer import MutationCoverageFuzzer

## Pintools
Pintools are _plugins_ to Pin, which control the type of instrumentation to be applied. A pintool's `main` function is responsible to inform Pin about which instrumentation functions should be used when JITing application code. Pintools are written in C++ and compiled as shared objects to be used by pin.

The [instruction counter](../../../edit/fuzzingbook/dbi/inscount.cpp) is a simple example of a pintool. Click on the link and have a look at the source code. You can edit the code directly in the browser.

We can build pintools from source using Pin's make system.

In [ ]:
!cd ../dbi/; PIN_ROOT=/usr/local/share/pin make tools

Now we can use our newly built pintool to count he instructions in any binary, say `ls`.

In [ ]:
!cd ../dbi/; pin -t ./obj-intel64/inscount.so -- ls

The pintool stored its output in `inscount.out`, which now contains the number of instructions executed by `ls`.

In [ ]:
%cat ../dbi/inscount.out

## Exercises
### Exercise 1: Coverage instrumentation with Pin

**Getting Coverage**

Create a pintool `coverage` that measures code coverage in a target binary. You can start from the `inscount` pintool. The build system is set up to expect a file `coverage.cpp` in the same directory as `inscount.cpp`. To build the tool, run make again.

In [ ]:
!cd ../dbi/; PIN_ROOT=/usr/local/share/pin make tools

Then run on a target program, e.g., the system-wide `ls`. No recompilation or `gcov` needed!

In [ ]:
!pin -t ../dbi/obj-intel64/coverage.so -- ls

You will notice that the addresses of basic blocks are different each time the program runs. This renders the results unusable for measuring coverage. 

We need to disable Address Space Layout Randomization temporarily when running Pin. This is possible using `setarch` and invoking pin as `setarch x86_64 -R pin -t ../dbi/obj-intel64/coverage.so -- ls`

**Feedback-driven Fuzzing**

Write a new `PinCoverageRunner` that runs a binary program using Pin and collects the coverage instrumentation. Then, like in the chapter on ["Mutation-based Fuzzing"](MutationFuzzer.ipynb), use a `MutationCoverageFuzzer` to fuzz `bc` (or another program) with your new runner class and plot the results.

In [ ]:
import subprocess

class PinCoverageRunner(ProgramRunner):

    def __init__(self, program):
        self.program = program

    def coverage(self):
        return self._coverage
    
    def run_process(self, inp=""):
        result = subprocess.run(['setarch', 'x86_64', '-R', 
                                 'pin', '-t', '../dbi/obj-intel64/coverage.so', '--', 
                                 self.program],
                              input=inp,
                              stdout=subprocess.PIPE,
                              stderr=subprocess.PIPE,
                              universal_newlines=True)
        self._coverage = []
        cov_file = 'coverage.out'
        with open(cov_file) as file:
            for line in file.readlines():
                line_number = int(line)
                self._coverage.append(('binary', line_number))

        return result

In [ ]:
pinrunner = PinCoverageRunner('bc')
mutation_fuzzer = MutationCoverageFuzzer(min_mutations=1, max_mutations=3, seed=['', 'f', '2+2\n'])
results = mutation_fuzzer.runs(pinrunner, trials=100)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(mutation_fuzzer.cumulative_coverage)
plt.title('Coverage of bc with MutationCoverageFuzzer')
plt.xlabel('# of inputs')
plt.ylabel('basic blocks covered');